In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as snc
from scipy.optimize import curve_fit
import datetime as dt
import calendar
from dateutil.relativedelta import relativedelta
from matplotlib.gridspec import GridSpec
plt.style.use('dark_background')

# Task 2: Storage contract pricing

In [5]:
# load our gas price model and parameters
popt = pd.read_csv('gas_price_model_parameters.csv')['model_parameters'].to_list()
data = pd.read_csv('Nat_Gas_formatted.csv')
data['Dates'] = pd.to_datetime(data['Dates'])

def PREDICT_GAS_PRICE(date):
    date = pd.to_datetime(date)
    def line_and_sin(x, a,b,c,d,e):
        return a + b*x + c*np.sin(2*np.pi*(d*x) + e)

    if date in data['Dates'].values: return data.loc[data['Dates'].values == date, 'Prices']
    else:
        months_elapsed_fromfirstdata = (date - data['Dates'].min()).days/30.4
        return line_and_sin(months_elapsed_fromfirstdata, *popt)

#let's save the optimal parameters to use for task 2 as a csv
fit_parameters = pd.DataFrame({'model_parameters': popt})
fit_parameters.to_csv('gas_price_model_parameters.csv')
data.to_csv('Nat_Gas_formatted.csv')
#let's save the optimal parameters to use for task 2 as a csv
fit_parameters = pd.DataFrame({'model_parameters': popt})
fit_parameters.to_csv('gas_price_model_parameters.csv')

print(PREDICT_GAS_PRICE('2025/10/31'))

12.9348972181626


In [29]:
#in this first toy, the entirety of the gas injected in one date is withdrawn on the next
def contract_price(injection_dates, withdrawal_dates, vol, injection_cost_per_MMBtu, charge_for_movement, monthly_storage_cost, max_possible_storage = 1e07):
    """Toy model for gas storage contract price
    
    injection and extraction dates are already ordered"""
    contract_price = np.zeros(len(injection_dates))

    # date_register = {'date_in':[], 'date_out':[], 'vol_in':[], 'vol_out':[] ,'type':[], 'volume':[] }
    for i in range(len(injection_dates)):

        if vol[i] > max_possible_storage: raise Exception(" This contract exceeds the facility's storage capacity")
  
        gas_price_at_injection = PREDICT_GAS_PRICE(injection_dates[i])
        print(f'inject {vol[i]} MMBtu at USD {gas_price_at_injection} per MMBtu + fees')
        injection_variable_cost = vol[i]*(gas_price_at_injection + injection_cost_per_MMBtu)
        total_injection_cost = injection_variable_cost + charge_for_movement
        print(f'Total injection cost:  USD {total_injection_cost}')
        contract_price[i] -= total_injection_cost

        gas_price_at_withdrawal = PREDICT_GAS_PRICE(withdrawal_dates[i])
        print(f'withdraw {vol[i]} MMBtu at USD {gas_price_at_withdrawal} per MMBtu + fees')
        withdrawal_variable_earnings = vol[i]*(gas_price_at_withdrawal - injection_cost_per_MMBtu)
        total_withdrawal_earnings = withdrawal_variable_earnings - charge_for_movement
        print(f'Total withdrawal earnings:   {total_withdrawal_earnings}')
        contract_price[i] += total_withdrawal_earnings

        #we need to account for the cost of keeping the storage tanks filled
        storage_time = (pd.to_datetime(withdrawal_dates[i]) - pd.to_datetime(injection_dates[i])).days
        months_stored = int(np.round(storage_time /30.4))
        storage_cost = months_stored*monthly_storage_cost
        print(f'Storage fees for {months_stored} months:    USD {storage_cost}')
        contract_price[i] -= storage_cost
        print(f'Value of leg {i+1} of the contract:   {contract_price[i]} ')

    contract_price = np.sum(contract_price)
    print(f'Contract Price: USD {contract_price}')
    
    return contract_price
    
contract_price(['2024/06/12', '2025/07/12' ],['2024/12/24', '2025/11/30'], [1e06, 3e06], 0.5, 100, 1000 )




inject 1000000.0 MMBtu at USD 11.644154237094247 per MMBtu + fees
Total injection cost:  USD 12144254.237094248
withdraw 1000000.0 MMBtu at USD 12.985339548016944 per MMBtu + fees
Total withdrawal earnings:   12485239.548016943
Storage fees for 6 months:    USD 6000
Value of leg 1 of the contract:   334985.3109226953 
inject 3000000.0 MMBtu at USD 12.038647870344471 per MMBtu + fees
Total injection cost:  USD 37616043.61103342
withdraw 3000000.0 MMBtu at USD 13.314657001640436 per MMBtu + fees
Total withdrawal earnings:   38443871.00492131
Storage fees for 5 months:    USD 5000
Value of leg 2 of the contract:   822827.3938878924 
Contract Price: USD 1157812.7048105877


1157812.7048105877

This toy program is simple but useful for gas storage contracts where tanks must be fully filled/emptied at every movement. If we can inject a fraction of the tank, we'd need to changt the part of the program that deals with monthly costs for storage and the checker for whether we are below the storage limit. This should be sufficient to perform quick calculations for a client.